In [ ]:
import sys
from joblib import load
from matplotlib import pyplot as plt
import numpy as np
import gurobipy as gp

In [ ]:
# import my functions
%load_ext autoreload
%autoreload 2
sys.path.append('../../src')
from ml2grb.extra.morerelu import reluOBBT
from ml2grb.extra.obbt import obbt
from ml2grb.activations2grb import Identity
from ml2grb.sklearn2grb import Pipe2Gurobi

In [ ]:
# Load the trained network and the examples
pipe = load('MNIST_50_50.joblib')
X = load('MNIST_first100.joblib')

# Choose an example
exampleno = 19
example = X[exampleno, :]
ex_prob = pipe.predict_proba(example.reshape(1, -1))
output_shape = ex_prob.shape

In [ ]:
pixels = example.reshape((28, 28))
plt.imshow(pixels, cmap='gray')
plt.show()

In [ ]:
pipe.predict(example.reshape(1, -1))

In [ ]:
sortedidx = np.argsort(ex_prob)[0]

In [ ]:
m = gp.Model()
epsilon = 7

x = m.addMVar(example.shape, lb=0.0, ub=1.0, name='X')
absdiff = m.addMVar(example.shape, lb=0, ub=1, name='dplus')
output = m.addMVar(output_shape, lb=-gp.GRB.INFINITY, name='y')

In [ ]:
m.setObjective(output[0, sortedidx[-2]] - output[0, sortedidx[-1]],
               gp.GRB.MAXIMIZE)

# Bound on the distance to example in norm-2
m.addConstr(absdiff >= x - example)
m.addConstr(absdiff >= - x + example)
m.addConstr(absdiff.sum() <= epsilon)

In [ ]:
# Code to add the neural network to the constraints
pipe2grb = Pipe2Gurobi(pipe, m)
# For this example we should model softmax in the last layer using identity
pipe2grb.steps[-1].actdict['softmax'] = Identity()
pipe2grb.predict(x, output)

In [ ]:
# Gurobi should do something internally about this
obbt(pipe2grb.steps[-1], activation=reluOBBT('both'))

In [ ]:
m.Params.OutputFlag = 1
m.Params.TimeLimit = 250
m.optimize()

In [ ]:
pixels = x.X.reshape((28, 28))
plt.imshow(pixels, cmap='gray')
plt.show()

In [ ]:
pipe.predict(x.X.reshape(1, -1))

In [ ]:
pipe.predict_proba(x.X.reshape(1, -1)).round(3)

pipe2grbght © 2022 Gurobi Optimization, LLC